In [3]:
import numpy as np
from numpy.random import rand
import math
from random import randint
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

In [157]:
class ConnectN:
    
    def __init__(self, grid_size, n):
        self.n = n
        self.grid_size = grid_size
        self.grid = np.zeros([grid_size,grid_size])
        self.finished = 0
        self.turn_num = 0
        
    def reset(self):
        self.__init__(self.grid_size, self.n)

    def check_win(self, col, row, player):
        for i in range(0, self.n):
            if sum(self.grid[col, row - i:row - i + self.n]) == self.n*player:
                self.finished = 1
                return 1
            if sum(self.grid[col - i: col - i + self.n, row]) == self.n*player:
                self.finished = 1
                return 1
            if col - i >= 0 and col - i + self.n - 1 < self.grid_size and row - i >= 0 and row - i + self.n - 1 < self.grid_size:
                if sum([self.grid[col - i + x, row - i + x] for x in range(0, self.n)]) == self.n*player:
                    self.finished = 1
                    return 1
            if col - i >= 0 and col - i + self.n - 1 < self.grid_size and row + i >= self.n - 1 and row + i < self.grid_size:
                if sum([self.grid[col - i + x, row + i - x] for x in range(0, self.n)]) == self.n*player:
                    self.finished = 1
                    return 1
        return 0

    def move(self, col, player):
        
        self.turn_num += 1
        
        if self.finished == 1:
            return 1, 50
        sum_col = np.sum([abs(x) for x in self.grid[col]])
        if sum_col == self.grid_size:
            return -1, -1
        self.grid[col, sum_col] = player
        if self.check_win(col, sum_col, player) == 1:
            return 1, 50
        return 0, 0
    
    def turn(self):
        """
        Returns which player's turn it is. First turn is player 1, second turn is player -1.
        """
        if self.turn_num%2 == 0:
            return 1
        else:
            return -1
        
    def next_possible_moves(self):
        """
        Returns array of possible columns for a next move
        """
        columns = []
        
        for i in xrange(0, self.grid_size):
            if (0 in self.grid[i]):
                columns.append(i)
                
        return columns
    
    def all_tokens_placed(self):
        """
        Returns location of all tokens (column, row) that have been placed
        """
        all_tokens = []
        
        for col in xrange(0, self.grid_size):
            for row in xrange(0, self.grid_size): 
                if self.grid[col][row] != 0:
                    all_tokens.append({"location": [col, row], "player": self.grid[col][row]})
                    
        return all_tokens
    
    def is_empty(self, col, row):
        return self.grid[col][row] == 0
    
    def print_grid(self):
        print(np.rot90(self.grid))

In [158]:
x = ConnectN(7, 5)
x.print_grid ()

[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]]


In [159]:
x.next_possible_moves()

[0, 1, 2, 3, 4, 5, 6]

In [160]:
x.all_tokens_placed()

[]

In [161]:
x.move(3, 1)

(0, 0)

In [162]:
x.print_grid()

[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.]]


In [163]:
x.all_tokens_placed()

[{'location': [3, 0], 'player': 1.0}]

In [164]:
x.is_empty(1, 0)

True

In [165]:
x.grid[1][0]

0.0

### Minimax

In [166]:
import copy

In [203]:
class Minimax_Learner(object):
    """
    Simple implementation of AI algorithm Minimax with static evaluator 
    """
    
    def __init__(self, board, depth, n, player):
        self.board = board
        self.depth = depth
        self.num_states = board.grid_size
        self.player = player
        self.n = n
        
        
    def streakVertical(self, board, col, row, player):
        if row > len(board[col]) - self.n:
            return 0
        for i in range(0,self.n):
            if board[col][row + i] == -1*player:
                return 0
            if board[col][row + i] == 0:
                return i
        return self.n

    def streakHorizontal(self, board, col, row, player):
        if col > len(board) - self.n:
            return 0
        for i in range(0,self.n):
            if board[col + i][row] == -1*player:
                return 0
            if board[col + i][row] == 0:
                return i
        return self.n
    
    def streakDiagonalUp(self, board, col, row, player):
        if row > len(board[col]) - self.n or col > len(board) - self.n:
            return 0
        for i in range(0,self.n):
            if board[col + i][row + i] == -1*player:
                return 0
            if board[col + i][row + i] == 0:
                return i
        return self.n
    
    def streakDiagonalDown(self, board, col, row, player):
        if row < self.n or col > len(board) - self.n:
            return 0
        for i in range(0,self.n):
            if board[col + i][row - i] == -1*player:
                return 0
            if board[col + i][row - i] == 0:
                return i
        return self.n

    def value(self, board):
        val = 0
        conversion = [int(math.pow(2, i))/2 for i in range(0, self.n+1)]
        conversion[self.n] = 200000000
        conversion_other = [int(math.pow(2, i))/4 for i in range(0, self.n+1)]
        conversion_other[self.n-1] = 1000000
        for i in range(0, len(board)):
            for j in range(0, len(board[0])):
                val += conversion[self.streakVertical(board, i, j, self.player)]
                val += conversion[self.streakHorizontal(board, i, j, self.player)]
                val += conversion[self.streakDiagonalUp(board, i, j, self.player)]
                val += conversion[self.streakDiagonalDown(board, i, j, self.player)]
                val -= conversion_other[self.streakVertical(board, i, j, -1*self.player)]
                val -= conversion_other[self.streakHorizontal(board, i, j, -1*self.player)]
                val -= conversion_other[self.streakDiagonalUp(board, i, j, -1*self.player)]
                val -= conversion_other[self.streakDiagonalDown(board, i, j, -1*self.player)]
        return val
        
    def create_tree(self, node, depth, player, move):
        if depth == 0:
            return None
        
        else:
            tree = {"value": 0, "children": [], "board": node, "player": player, "move": move}

            next_moves = node.next_possible_moves()

            for move in next_moves:
                board_copy = copy.deepcopy(node)
                board_copy.move(move, self.player)
                new_child = self.create_tree(board_copy, depth-1, 1-player, move)
                
                if new_child != None:
                    tree["children"].append(new_child)

            return tree

    def children(self, node):
        """ 
        returns children of a node
        """
        return node["children"]
   
    def leaf(self, node):
        """
        returns if current node is a leaf (i.e. no children)
        """
        return len(self.children(node)) == 0
        
    def max_node(self, node):
        """
        returns true if node is a max node and false if a node
        is a min node
        """
        return node["player"] == self.player
        
    def evaluate(self, node):
        """
        Static evaluator function to return a value between Loss and Win for intermediate game
        positions, larger if the position is better for the current player.
        If depth limit of the search is exceeded, is applied to remaining nodes as if
        they were leaves. 
        
        We calculate the rating by:
        1. Checking each token already placed, and checking how many possible ways to connect N there are
        2. Weight this by how complete these win paths are
        #3. If tokens in win streak are our player, add 1 point
        #4. If tokens in win streak are other player, subtract 1 point
        """
        node["value"] = self.value(node["board"].grid)
        return node["value"]       

    def minimax(self, node, depth):
        """ 
        Recursive implementation of Minimax algorithm using pseudocode from: 
        https://www.cs.cornell.edu/courses/cs312/2002sp/lectures/rec21.htm
        
        Loss and Win values should be set to - and + infinity, respectively, but
        here use -/+ 1000
        """
        if self.leaf(node) or depth == 0:
            return self.evaluate(node)
        
        if self.max_node(node):
            # L = -infinity
            current_node_value = -1000000000
            for child in self.children(node):
                next_node_value = self.minimax(child, depth-1)
                if current_node_value < next_node_value:
                    current_node_value = next_node_value
            node["value"] = current_node_value
            return current_node_value
        
        if not self.max_node(node):
            # W = +infinity
            current_node_value = 10000000000
            for child in self.children(node):
                next_node_value = self.minimax(child, depth-1)
                if next_node_value < current_node_value:
                    current_node_value = next_node_value
            node["value"] = current_node_value
            return current_node_value

    def calc_next_move(self):
        current_tree = self.create_tree(self.board, self.depth, self.player, None)
        top_val = self.minimax(current_tree, self.depth)
        print "this is top_val", top_val
                
        #print current_tree- works
        for child in current_tree["children"]:
            if child["value"] == top_val:
                print "i'm here"
                return child["move"]
        
        top_val = np.min([x["value"] for x in current_tree["children"]])
        for child in current_tree["children"]:
            if child["value"] == top_val:
                print "i'm here"
                return child["move"]
        
        

In [236]:
class Random_Learner(object):
    """
    Simple implementation of random algorithm
    """
    
    def __init__(self, board):
        self.board = board
        

    def calc_next_move(self):
        moves = self.board.next_possible_moves()
        return moves[random.randint(0, len(moves) - 1)]
        

In [237]:
def play_game(x, p1, p2):    
    while True:
        print("p1")
        p1move = p1.calc_next_move()
        print(p1move)
        if (p1move is None):
            x.print_grid()
            print("error")
            return -1
        p1result = x.move(p1move, 1)
        print p1result
        if (p1result[0] == 1):
            print("player 1")
            x.print_grid()
            return 1
        elif (p1result[0] == -1):
            x.print_grid()
            print("error")
            return -1
        print("p2")
        p2move = p2.calc_next_move()
        print(p2move)
        if (p2move is None):
            x.print_grid()
            print("error")
            return -1
        p2result = x.move(p2move, -1)
        print p2result
        if (p2result[0] == 1):
            print("player 2")
            x.print_grid()
            return 1
        elif (p2result[0] == -1):
            x.print_grid()
            print("error")
            return -1

In [239]:
x = ConnectN(7, 4)
p1 = Minimax_Learner(x, 3, 4, 1) 
p2 = Random_Learner(x)
play_game(x, p1, p2)

p1
this is top_val 4
i'm here
0
(0, 0)
p2
5
(0, 0)
p1
this is top_val 8
i'm here
0
(0, 0)
p2
5
(0, 0)
p1
this is top_val 13
i'm here
0
(0, 0)
p2
6
(0, 0)
p1
this is top_val 200000014
i'm here
0
(1, 50)
player 1
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0. -1.  0.]
 [ 1.  0.  0.  0.  0. -1. -1.]]


1

In [241]:
x = ConnectN(7, 4)
p1 = Minimax_Learner(x, 3, 4, 1) 
p2 = Random_Learner(x) 
play_game(x, p1, p2)

p1
this is top_val 4
i'm here
0
(0, 0)
p2
3
(0, 0)
p1
this is top_val 8
i'm here
0
(0, 0)
p2
4
(0, 0)
p1
this is top_val 13
i'm here
0
(0, 0)
p2
2
(0, 0)
p1
this is top_val 199000014
i'm here
0
(1, 50)
player 1
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.]
 [ 1.  0. -1. -1. -1.  0.  0.]]


1

In [240]:
x = ConnectN(7, 4)
p1 = Minimax_Learner(x, 3, 4, 1) 
p2 = Minimax_Learner(x, 3, 4, -1) 
play_game(x, p1, p2)

p1
this is top_val 4
i'm here
0
(0, 0)
p2
this is top_val 4
i'm here
0
(0, 0)
p1
this is top_val 7
i'm here
1
(0, 0)
p2
this is top_val 7
i'm here
0
(0, 0)
p1
this is top_val 12
i'm here
2
(0, 0)
p2
this is top_val 11
i'm here
3
(0, 0)
p1
this is top_val 10
i'm here
1
(0, 0)
p2
this is top_val 14
i'm here
0
(0, 0)
p1
this is top_val 13
i'm here
0
(0, 0)
p2
this is top_val 13
i'm here
1
(0, 0)
p1
this is top_val 14
i'm here
2
(0, 0)
p2
this is top_val 18
i'm here
2
(0, 0)
p1
this is top_val -999985
i'm here
3
(0, 0)
p2
this is top_val 199000018
i'm here
3
(1, 50)
player 2
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.]
 [-1.  0.  0.  0.  0.  0.  0.]
 [-1. -1. -1. -1.  0.  0.  0.]
 [-1.  1.  1.  1.  0.  0.  0.]
 [ 1.  1.  1. -1.  0.  0.  0.]]


1